In [1]:
import numpy as np
from pysrc.optimization import gams, gurobi
from pysrc.sampling import baseline
from pysrc.services.data_service import load_site_data
from pysrc.analysis import value_decomposition

/Users/ph2696/Projects/project-amazon/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Model scenario

In [2]:
opt = "gurobi"
pee = 7.1
pa = 41.11
b = 25
sitenum = 78
timehzn = 200

## Solve model

In [3]:
# Load site data
(
    zbar_2017,
    z_2017,
    forest_area_2017,
    _,
    _,
    _,
    _,
) = load_site_data(sitenum)

# Set productivity parameters using baseline mean
baseline_fit = baseline.sample(
    model_name="full_model",
    num_sites=sitenum,
    iter_sampling=10**4,
    chains=5,
    seed=1,
)

theta = baseline_fit.stan_variable("theta").mean(axis=0)
gamma = baseline_fit.stan_variable("gamma").mean(axis=0)

# Computing carbon absorbed in start period
x0_vals = gamma * forest_area_2017

# Choose optimizer
if opt == "gurobi":
    solve_planner_problem = gurobi.solve_planner_problem

elif opt == "gams":
    solve_planner_problem = gams.solve_planner_problem

else:
    raise ValueError("Optimizer must be one of ['gurobi', 'gams']")

results = solve_planner_problem(
    T=timehzn,
    theta=theta,
    gamma=gamma,
    x0=x0_vals,
    zbar=zbar_2017,
    z0=z_2017,
    pe=pee + b,
    pa=pa,
)

Data successfully loaded from /Users/ph2696/Projects/project-amazon/data/hmc


In [ ]:
results["X"].shape

In [ ]:
results["X"].sum(axis=1)

In [ ]:
value_decomposition(
    Z=results["Z"],
    X=results["X"],
    U=results["U"],
    V=results["V"],
    T=timehzn,
    pee=pee,
    pa=pa,
    b=b,
    theta=theta,
)

In [ ]:
V = [
    value_decomposition(
        Z=results["Z"][t:],
        X=results["X"][t:],
        U=results["U"][t:],
        V=results["V"][t:],
        T=200,
        pee=pee,
        pa=pa,
        b=b,
        theta=theta,
    )["total_PV"]
    for t in range(50)
]

In [ ]:
import matplotlib.pyplot as plt

# Plotting the line plot
plt.plot(V)

# Adding labels and title
plt.xlabel("Time")
plt.ylabel("Value")
plt.title("V(t)")

# Displaying the plot
plt.show

In [ ]:
W = []
for t in range(50):
    defection = solve_planner_problem(
        T=200,
        theta=theta,
        gamma=gamma,
        x0=results["X"][t],
        zbar=zbar_2017,
        z0=results["Z"][t],
        pe=pee,
        pa=pa,
    )

    W.append(
        value_decomposition(
            Z=defection["Z"],
            X=defection["X"],
            U=defection["U"],
            V=defection["V"],
            T=200,
            pee=pee,
            pa=pa,
            b=0,
            theta=theta,
        )["total_PV"]
    )

In [ ]:
# Plotting the line plots for V and W
plt.plot(V, label="V(t)")
plt.plot(W, label="W(t)")

# Adding legend
plt.legend()

# Adding labels and title
plt.xlabel("Time")
plt.ylabel("Value")
plt.title("Value of reforestation v.s defecting")

# Displaying the plot
plt.show()

In [ ]:
(1 / 1.02) ** 50

In [ ]:
(V[-1] - W[-1]) * ((1 / 1.02) ** 50)